In [2]:
import pandas as pd
import tomllib

from data.data_processing import select_rename_columns

In [ ]:
CONFIG_PATH = './data/config.toml'
def load_config(section_name: str) -> dict:
    with open(CONFIG_PATH, 'rb') as config_file:
        return tomllib.load(config_file)[section_name]

In [ ]:
df = pd.read_csv('./data/raw_data/pop_census/base-cc-serie-historique-2020.CSV', sep=';', dtype='object')

def population_preprocessing(df: pd.DataFrame, table: str='statistiques_pop') -> pd.DataFrame:
    population_config = load_config(table)

    df = df.copy()[~df['CODGEO'].str.startswith('97')]

    rename_mapper = {key: value for key, value in population_config.items() if isinstance(value, str)}
    df = df[list(population_config.keys())].rename(
        columns=rename_mapper
    )
    
    temp_dfs = []
    statistics_mapper = {key: value for key, value in population_config.items() if isinstance(value, list)}
    nbr_max_params = sum(map(len, statistics_mapper.values()))
    for column, fields in statistics_mapper.items():
        temp_dfs.append(
            pd.DataFrame(
                {name: df[name] for name in rename_mapper.values()} | {
                    'annee_debut': fields[1],
                    'annee_fin': fields[1] if len(fields) < nbr_max_params else fields[2],
                    'type_statistique': fields[0],
                    'valeur': df[column].astype(float)
                }
            )
        )

    return pd.concat(temp_dfs, ignore_index=True)

population_preprocessing(df)

In [18]:
df = pd.read_csv('./data/raw_data/area/v_commune_2023.csv', dtype='object')
df1 = pd.read_csv('./data/raw_data/pop_census/base-cc-serie-historique-2020.CSV', sep=';', dtype='object')


df1[~df1['CODGEO'].isin(df[df['TYPECOM'] == 'ARM']['COM'])]

,CODGEO,P20_POP,P14_POP,P09_POP,D99_POP,D90_POP,D82_POP,D75_POP,D68_POP,SUPERF,...,D75_LOGVAC,D68_LOGVAC,P20_PMEN,P14_PMEN,P09_PMEN,D99_PMEN,D90_NPER_RP,D82_NPER_RP,D75_NPER_RP,D68_NPER_RP
0,01001,806,767,787,728,579,477,368,347,15.95,...,22,25,806,767,787,728,579,477,368,347
1,01002,262,239,207,168,159,136,106,88,9.15,...,23,1,262,239,207,168,159,136,106,88
2,01004,14288,14022,13350,11436,10455,9737,9550,8949,24.6,...,246,155,13787.0089845858,13547.0112272758,12781.9983314098,11015,10092,9333,9164,8582
3,01005,1782,1627,1592,1408,1156,848,756,627,15.92,...,16,15,1782,1627,1592,1406,1155,848,753,627
4,01006,113,109,120,86,76,65,82,108,5.88,...,13,8,113,109,120,86,76,65,82,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34968,97420,24086,22406,22437,18144,14695,13196,11984,11151,57.84,...,267,159,23980.9997840365,22336.0001378959,22387.9993355204,18135,14685,13154,11965,11041
34969,97421,7310,7132,7406,7402,7004,6467,6462,7105,103.82,...,201,138,7307.00011307918,7129,7312,7372,6973,6435,6448,6881
34970,97422,80778,76796,72658,60323,47593,40545,36107,31378,165.43,...,0,617,80339.6690309185,76408.0672664655,72219.049155679,60123,47460,39741,36001,30416
34971,97423,6953,7198,7057,6598,5767,5132,4864,4471,42.58,...,113,86,6953,7188.99958319828,7052.00023616098,6597,5763,5132,4864,4465
